# Load the Model

In [ ]:
import numpy as np
import azureml
from azureml.core import Workspace, Run
from azureml.core.model import Model

print("Azure ML: ", azureml.core.VERSION)
ws = Workspace.from_config()
model = Model(ws, "quality-prediction-gbm.pkl")

# Create YAML file for Dependencies

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies.create(conda_packages=['numpy','scikit-learn'],pip_packages=['azureml-sdk[automl]'])

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

# Create the Model Image and Webservice

In [ ]:
from azureml.core.image import ContainerImage
image_config = ContainerImage.image_configuration(execution_script = "score.py",
                                                  runtime = "python",
                                                  conda_file = "myenv.yml",
                                                  dependencies = ['quality-prediction-gbm.pkl'],
                                                  description = "",
                                                  tags = {"type": "qualityprediction-gbm"})


from azureml.core.webservice import AciWebservice
aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 2, 
                                               tags = {"type": "qualityprediction-gbm"},
                                               description = '')

from azureml.core.webservice import Webservice

service = Webservice.deploy_from_model(deployment_config = aciconfig,
                                       image_config = image_config,
                                       models = [model], # this is the registered model object
                                       name = 'qualityprediction-gbm-aci-1',
                                       workspace = ws)
service.wait_for_deployment(show_output = True)

In [ ]:
print("Scoring API Url => ",service.scoring_uri)

#print(service.state)
#service.get_logs()